In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U kaleido # for export image https://github.com/plotly/Kaleido

import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from kaleido.scopes.plotly import PlotlyScope 
# import plotly.io as pio
# pio.kaleido.scope.default_format = 'pdf' # https://plotly.com/python/static-image-export/

%cd '/content/drive/MyDrive/Colab Notebooks/media-agenda/'
from util import OptimalKClustersConfig

%cd '/content/drive/MyDrive/Colab Notebooks/media-agenda/plot'

     |████████████████████████████████| 74.6MB 54kB/s 
/content/drive/.shortcut-targets-by-id/1jko4SIvsKFTxv155O0DV1wHZ_jS75VfX/Colab Notebooks/media-agenda
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/content/drive/.shortcut-targets-by-id/1jko4SIvsKFTxv155O0DV1wHZ_jS75VfX/Colab Notebooks/media-agenda/plot


# Parallel Categories Diagrams

## For k = {5, 10, 15, 20}

In [17]:
topics_k_5 = ['Policy & compliance', 'Garbage', 'Garbage', 'Organic vs. conventional farming', 'Retailers & prices & quality']
topics_k_10 = ['Food products & taste', 'Garbage', 'Garbage', 'Organic vs. conventional farming', 'Food safety', 'Politics & Policy & Compliance', 'Garbage', 'Retailers & prices', 'Health & nutrition', 'GMO & organic']
topics_k_15 = ['Environment (pesticides & fertilizers)', 'Retailers', 'Garbage', 'GMO & organic', 'Garbage', 'Food products & taste', 'Food safety', 'Research', 'Health & nutrition', 'Garbage', 'Politics & policy & compliance', 'Animal welfare & meat consumption', 'Farming & agricultural policy & food security', 'Consumer prices & profit', 'Garbage']
topics_k_20 = ['Sustainable farming', 'Research & scientific evidence', 'Garbage', 'GMO & organic', 'Consumer prices', 'Environment & pesticides & chemicals', 'Food prices & quality', 'Garbage', 'Animal welfare & meat consumption', 'Genetic research & breeding', 'Politics', 'Health & nutrition', 'Retailers', 'Garbage', 'Garbage', 'Environmental pollution', 'Food safety', 'Consumers & markets & corporations', 'Food products & taste', 'Compliance']

topics_for_cluster = {5: topics_k_5,
                      10: topics_k_10,
                      15: topics_k_15,
                      20: topics_k_20}

In [18]:
K = [5, 10, 13, 14, 15, 16, 20]
categoryarray = {5: [4, 3, 0, 2, 1],
                 10: [0, 8, 9, 4, 3, 7, 5, 6, 2, 1],
                 13: [4, 0, 9, 1, 10, 12, 7, 3, 11, 8, 5, 2, 6], 
                 14: [0, 4, 8, 9, 5, 10, 6, 1, 11, 12, 2, 3, 13, 7],
                 15: [5, 11, 8, 1, 3, 6, 0, 12, 13, 10, 7, 14, 2, 9, 4],
                 16: [11, 13, 4, 14, 15, 3, 2, 7, 8, 10, 5, 6, 12, 0, 1, 9],
                 20: [18, 8, 11, 6, 12, 3, 15, 16, 5, 0, 4, 17, 10, 1, 9, 19, 7, 13, 14, 2]}


labels_of_5_clusters = np.load('/content/drive/MyDrive/Colab Notebooks/media-agenda/refactorization/data/kmean-result-backup/5_kmean_labels.npy')
labels_of_10_clusters = np.load('/content/drive/MyDrive/Colab Notebooks/media-agenda/refactorization/data/kmean-result-backup/10_kmean_labels.npy')
labels_of_13_clusters = np.load('/content/drive/MyDrive/Colab Notebooks/media-agenda/refactorization/data/13_kmean_labels.npy')
labels_of_14_clusters = np.load('/content/drive/MyDrive/Colab Notebooks/media-agenda/refactorization/data/14_kmean_labels.npy')
labels_of_15_clusters = np.load('/content/drive/MyDrive/Colab Notebooks/media-agenda/refactorization/data/15_kmean_labels.npy')
labels_of_16_clusters = np.load('/content/drive/MyDrive/Colab Notebooks/media-agenda/refactorization/data/16_kmean_labels.npy')
labels_of_20_clusters = np.load('/content/drive/MyDrive/Colab Notebooks/media-agenda/refactorization/data/kmean-result-backup/20_kmean_labels.npy')

df = pd.DataFrame({'labels_of_5_clusters': labels_of_5_clusters, 
                   'labels_of_10_clusters': labels_of_10_clusters,
                   'labels_of_13_clusters': labels_of_13_clusters,
                   'labels_of_14_clusters': labels_of_14_clusters,
                   'labels_of_15_clusters': labels_of_15_clusters,
                   'labels_of_16_clusters': labels_of_16_clusters,
                   'labels_of_20_clusters': labels_of_20_clusters
                   })

df['count'] = 1
count_df = df.groupby(by = ['labels_of_5_clusters', 'labels_of_10_clusters', 
                            'labels_of_13_clusters', 'labels_of_14_clusters', 
                            'labels_of_15_clusters', 'labels_of_16_clusters', 'labels_of_20_clusters']).count().reset_index()
df = df.drop(['count'], axis = 1)

pc_df = count_df.merge(df, how = 'right', on = ['labels_of_5_clusters', 'labels_of_10_clusters', 
                                                'labels_of_13_clusters', 'labels_of_14_clusters', 
                                                'labels_of_15_clusters', 'labels_of_16_clusters', 'labels_of_20_clusters'], validate = 'one_to_many')

def show_parallel_categories_diagrams(pc_df, threshold = 0, categoryarray = None, title = None, width = 1000, height = 1000, margin_left = 200, margin_right = 200):
  def reorder(arr, index):
    result = []
    for i in index:
      result.append(arr[i])

    return result
  

  # Create dimensions
  dimensions = []
  for k in K:

    garbage_cluster_indeces = [i for i, v in enumerate(topics_for_cluster[k]) if 'Garbage' in v]
    garbage_precentage = str(round(df[df['labels_of_' + str(k) + '_clusters'].isin(garbage_cluster_indeces)].shape[0] / df['labels_of_' + str(k) + '_clusters'].shape[0] * 100, 2)) + '%'

    if k >= 5 and k <= 20:
      dimensions.append(
        go.parcats.Dimension(
          values = pc_df[pc_df['count'] > threshold]['labels_of_' + str(k) + '_clusters'],
          label = 'k = ' + str(k) + ' [' + garbage_precentage + ']',
          categoryorder = 'array',
          categoryarray = np.arange(0, k + 1) if categoryarray is None else categoryarray[k],
          ticktext = topics_for_cluster[k] if categoryarray is None else reorder(topics_for_cluster[k], categoryarray[k]),
        )
      )
    else:
      dimensions.append(
        go.parcats.Dimension(
          values = pc_df[pc_df['count'] > threshold]['labels_of_' + str(k) + '_clusters'],
          label = 'k = ' + str(k) + ' [' + garbage_precentage + ']',
          categoryorder = 'array',
          categoryarray = np.arange(0, k + 1) if categoryarray is None else categoryarray[k],
        )
      )

  # Create parcats trace
  color = pc_df[pc_df['count'] > threshold]['labels_of_' + str(K[0]) + '_clusters'];
  # colorscale = ['red', 'mediumseagreen', 'black', 'yellow', 'blue']
  colorscale = ['#86A3C3', '#DBE0BB', '#B6CEC7', '#7268A6', '#683074']

  data = [go.Parcats(dimensions = dimensions,
          line = {'color': color, 'colorscale': colorscale},
          # sortpaths = 'forward',
          hoveron = 'color', hoverinfo = 'count+probability',
          labelfont = {'size': 16, 'family': 'Times'},
          tickfont = {'size': 14, 'family': 'Times'},
          arrangement='freeform')]

  fig = go.Figure(data)
  
  # fig.update_layout(title_text = title)
  fig.update_layout(title_text = title, autosize = True, height = height, margin = dict(l = margin_left, r = margin_right))
  # fig.update_layout(title_text = title, autosize = False, width = width, height = height)

  fig.show()

  return data, fig

In [20]:
%pwd
plotly.offline.plot(data, image_filename='sankey_5_10_15_20.svg', image='svg')


'temp-plot.html'

In [19]:
K = [5, 10, 15, 20]
data, fig = show_parallel_categories_diagrams(pc_df, threshold = 40, categoryarray = categoryarray, width = 1500)

Output hidden; open in https://colab.research.google.com to view.